In [ ]:
<a href="https://colab.research.google.com/github/MihaiDogariu/CV3/blob/main/laborator/CV%203%20-%20Lab%20%237.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detector Moș Crăciun

Să se realizeze o aplicație care îl detectează pe Moș Crăciun în imagini.
Procesul, pentru fiecare imagine, este următorul:
1. se rulează detectorul de obiecte pe imaginea curentă;
1. se selectează doar casetele de încadrare/măștile obiectelor din clasa "person";
1. se calculează procentul de culoare roșie din fiecare casetă/mască (considerăm culoarea roșie toate nuanțele ce se încadrează în intervalul [210, 0, 0] - [255, 60, 60]; intervalul este flexibil și poate fi adaptat)
1. se calculează procentul de culoare albă din fiecare casetă/mască (considerăm culoarea albă toate nuanțele ce se încadrează în intervalul [220, 220, 220] - [255, 255, 255]; intervalul este flexibil și poate fi adaptat);
1. pe baza procentajului de culoare roșie și albă se determină dacă persoana respectivă este Moș Crăciun sau nu, e.g. Moș Crăciun se află în imaginile unde au fost depistate persoane, procentajul de culoare roșie din mască depășește 20%, iar cel de culoare albă depășește 15%.

Se va porni de la codul de mai jos (similar cu laboratorul precedent, însă detectorul este Mask R-CNN, în loc de Faster R-CNN).

# \#TODO:
1. implementați o funcție care calculează procentajul de culori roșu și alb dintr-o casetă de încadrare;
1. implementați o funcție care calculează procentajul de culori roșu și alb dintr-o mască de segmentare;
1. determinați empiric valorile pragurilor peste care trebuie să se situeze culorile respective pentru a-l detecta pe Moș Crăciun (în mod normal, procesarea casetelor va avea alte praguri față de procesarea măștilor);
1. în imaginea de ieșire păstrați culorile imaginii originale doar pe caseta de încadrare/masca în care a fost detectat Moș Crăciun - transformați restul imaginii în nuanțe de gri (imagine grayscale);
1. comparați rezultatele obținute cu cele 2 tipuri de procesări;
1. descărcați încă 4 imagini cu Moș Crăciun și testați-le cu valorile găsite.

In [ ]:
# se importa bibliotecile utilizate
%matplotlib inline
import matplotlib.pyplot as plt 
import torch
import torchvision.transforms as T
import torchvision
import numpy as np
import cv2
import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML
from PIL import Image

In [ ]:
# se descarca un numar aleator de imagini care contin persoane imbracate in Mos Craciun
!wget -nv https://static01.nyt.com/images/2020/11/21/us/00xp-santa-print/merlin_179905233_257c3550-7981-4341-b244-8f0e28a864c5-mobileMasterAt3x.jpg -O 1.jpg
!wget -nv https://media.cnn.com/api/v1/images/stellar/prod/211112151622-01-mall-santa-2020.jpg?q=x_38,y_233,h_1652,w_2937,c_crop/h_720,w_1280 -O 2.jpg
!wget -nv https://static.tvtropes.org/pmwiki/pub/images/mallsanta_9838.jpg -O 3.jpg
!wget -nv https://s.yimg.com/ny/api/res/1.2/l55xcp6q18xeWOE30sVL7A--/YXBwaWQ9aGlnaGxhbmRlcjt3PTk2MDtoPTU0MA--/https://media.zenfs.com/en/thedailybeast.com/4dde6d5f44312d8aed36c5f346e7c0d7 -O 4.jpg

In [ ]:
# se descarca modelul pre-antrenat: Faster R-CNN cu backbone ResNet-50 si Feature Pyramid Network
model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(pretrained=True)
# se seteaza modelul in starea de evaluare/inferenta - acest atribut schimba comportamentul unor straturi fata de modul in care se comporta in timpul antrenarii, e.g. batch normalization
model.eval()


# se incarca lista de etichete a bazei de date MS-COCO
COCO_INSTANCE_CATEGORY_NAMES = [
    'bg', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
# functie de detectie: citeste o imagine, o pre-proceseaza, o propaga prin model si intoarce rezultatele detectiei
def get_prediction(img_path, confidence_threshold):
  img = Image.open(img_path) # se deschide imaginea folosind calea sa absoluta ca argument
  transform = T.Compose([T.ToTensor()]) # se creeaza un set de transformari pe care le vom aplica fiecarei imagini citite - in acest caz, este suficienta doar transformarea imaginii in tensor
  img = transform(img) # se aplica transformarea pe imaginea deschisa
  pred = model([img]) # se propaga imaginea prin modelul de detectie
                      # pentru fiecare imagine din lista de intrare se returneaza un dictionar care contine:
                      # 1. lista de casete de incadrare
                      # 2. lista de clase detectate
                      # 3. lista de scoruri (confidence score) asociate detectiilor
                      # detectiile sunt plasate in lista de iesire in ordinea descrescatoare a scorului de detectie
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())] # se transforma lista de clase numerice detectate in lista de cuvinte asociate claselor conform bazei MS-COCO
  pred_boxes = [[int(i[0]), int(i[1]), int(i[2]), int(i[3])] for i in list(pred[0]['boxes'].detach().numpy())]  # se extrag casetele de incadrare 
                                                                                                                    # fiecare coordonata trebuie transformata manual in data de tipul int, deoarece, 
                                                                                                                    # in urma regresiei casetelor de incadrare, coordonatele sunt calculate ca valori reale
  pred_score = list(pred[0]['scores'].detach().numpy()) # se extrag scorurile de detectie
  pred_masks = list(pred[0]['masks'].detach().numpy().squeeze()) # se extrag mastile de detectie
  pred_t = [pred_score.index(x) for x in pred_score if x>confidence_threshold][-1] # se selecteaza indecsii detectiilor al caror scor depaseste pragul impus
  pred_boxes = pred_boxes[:pred_t+1] # se selecteaza doar casetele de incadrare ale obiectelor al caror scor depaseste pragul impus
  pred_class = pred_class[:pred_t+1] # se selecteaza doar clasele obiectelor al caror scor depaseste pragul impus
  pred_score = pred_score[:pred_t+1] # se selecteaza doar scorurile obiectelor care depasesc pragul impus
  pred_masks = pred_masks[:pred_t+1] # se selecteaza doar mastile obiectelor care depasesc pragul impus
  return pred_boxes, pred_class, pred_score, pred_masks

In [ ]:
# functie care apeleaza functia de detectie pentru o imagine si afiseaza rezultatele
def detect_object(img_path, confidence=0.5, rect_th=2, text_size=0.5, text_th=1):
  boxes, pred_cls, score, masks = get_prediction(img_path, confidence) # rularea detectiei
  # afisarea rezultatelor pe imaginea originala:
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  for i in range(len(boxes)):
    cv2.rectangle(img, (boxes[i][0], boxes[i][1]), (boxes[i][2], boxes[i][3]), color=(0, 255, 0), thickness=rect_th) # afisarea casetei de incadrare
    cv2.putText(img,"{}:{:.2f}".format(pred_cls[i], score[i]), (boxes[i][0], boxes[i][1]-6), cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # afisarea clasei si a scorului pentru fiecare caseta
  plt.figure(figsize=(20,30))
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()

In [ ]:
detect_object('3.jpg', confidence=0.7)